In [ ]:
!pip install -U transformers
import os
os.kill(os.getpid(), 9)  # Force restart to reload updated package


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 120.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [1]:
!pip install -q transformers datasets scikit-learn
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.5 MB/s eta 0:00:00


In [2]:
import transformers
print(transformers.__version__)


4.52.3


In [2]:
import pandas as pd

# Load dataset files
train_df = pd.read_csv("/content/drive/MyDrive/emoji dataset/Train.csv")
mapping_df = pd.read_csv("/content/drive/MyDrive/emoji dataset/Mapping.csv")

print(train_df.head())
print(mapping_df.head())


   Unnamed: 0                                               TEXT  Label
0           0  Vacation wasted ! #vacation2017 #photobomb #ti...      0
1           1  Oh Wynwood, you’re so funny! : @user #Wynwood ...      1
2           2  Been friends since 7th grade. Look at us now w...      2
3           3  This is what it looks like when someone loves ...      3
4           4  RT @user this white family was invited to a Bl...      3
   Unnamed: 0 emoticons  number
0           0         😜       0
1           1         📸       1
2           2         😍       2
3           3         😂       3
4           4         😉       4


In [3]:
# Rename columns in train_df for consistency
train_df.rename(columns={'TEXT': 'text', 'Label': 'label'}, inplace=True)

# Create a mapping dictionary from number to emoji (optional, for display)
label_to_emoji = dict(zip(mapping_df["number"], mapping_df["emoticons"]))

# Encode train_df["label"] as integer using mapping_df["number"]
# Ensure labels are integers
train_df["label"] = train_df["label"].astype(int)

print("Sample data after processing:")
print(train_df.head())

# Optional: view mapping dictionary
print("\nEmoji mapping (number → emoticon):")
print(label_to_emoji)


Sample data after processing:
   Unnamed: 0                                               text  label
0           0  Vacation wasted ! #vacation2017 #photobomb #ti...      0
1           1  Oh Wynwood, you’re so funny! : @user #Wynwood ...      1
2           2  Been friends since 7th grade. Look at us now w...      2
3           3  This is what it looks like when someone loves ...      3
4           4  RT @user this white family was invited to a Bl...      3

Emoji mapping (number → emoticon):
{0: '😜', 1: '📸', 2: '😍', 3: '😂', 4: '😉', 5: '🎄', 6: '📷', 7: '🔥', 8: '😘', 9: '❤', 10: '😁', 11: '🇺🇸', 12: '☀', 13: '✨', 14: '💙', 15: '💕', 16: '😎', 17: '😊', 18: '💜', 19: '💯'}


In [4]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Split data
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["text"].tolist(), train_df["label"].tolist(), test_size=0.2, random_state=42
)

# Tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
import torch

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmojiDataset(train_encodings, train_labels)
val_dataset = EmojiDataset(val_encodings, val_labels)


In [6]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["label"])

num_labels = len(label_encoder.classes_)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True,
    save_steps=500,
    save_total_limit=2
)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: harshithmanikhantasunkara3119 (harshithmanikhantasunkara3119-vellore-institute-of-techn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.020300
20,3.020900
30,3.021900
40,2.979300
50,2.951000
60,2.905800
70,2.850600
80,2.828200
90,2.784700
100,2.754200


TrainOutput(global_step=10500, training_loss=1.9060880207788378, metrics={'train_runtime': 1755.0259, 'train_samples_per_second': 95.725, 'train_steps_per_second': 5.983, 'total_flos': 4043623066560000.0, 'train_loss': 1.9060880207788378, 'epoch': 3.0})

In [8]:
trainer.evaluate(val_dataset)


{'eval_loss': 2.1730055809020996,
 'eval_runtime': 32.1746,
 'eval_samples_per_second': 435.126,
 'eval_steps_per_second': 27.195,
 'epoch': 3.0}

In [9]:
# Assuming you have:
# val_dataset  - validation dataset (tokenized + labels)
# trainer      - Trainer object used for training

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate(eval_dataset=val_dataset)

print(f"Validation Results: {eval_results}")

# This will print metrics including validation loss and accuracy if you set it in compute_metrics


Validation Results: {'eval_loss': 2.1730055809020996, 'eval_runtime': 30.2676, 'eval_samples_per_second': 462.54, 'eval_steps_per_second': 28.909, 'epoch': 3.0}


In [10]:
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.argmax(axis=1)


In [11]:
pred_emoji = label_encoder.inverse_transform(preds)


In [12]:
trainer.save_model("emoji-predictor-model")
tokenizer.save_pretrained("emoji-predictor-model")


('emoji-predictor-model/tokenizer_config.json',
 'emoji-predictor-model/special_tokens_map.json',
 'emoji-predictor-model/vocab.txt',
 'emoji-predictor-model/added_tokens.json',
 'emoji-predictor-model/tokenizer.json')

In [ ]:
import torch

# Function to predict emoji for user input
def predict_emoji(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Put model in eval mode
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()

    # Decode label back to emoji
    predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]
    predicted_emoji = label_to_emoji.get(predicted_label, "❓")

    return predicted_emoji


# 🔄 Loop for user input
while True:
    user_input = input("Enter a sentence (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    emoji = predict_emoji(user_input)
    print(f"Predicted Emoji: {emoji}")


Enter a sentence (or type 'exit' to quit): i really love you !
Predicted Emoji: ❤
Enter a sentence (or type 'exit' to quit): i am so happy today!
Predicted Emoji: 😊
Enter a sentence (or type 'exit' to quit): Happy birthday harshith
Predicted Emoji: ❤
Enter a sentence (or type 'exit' to quit): i am so angry right now.
Predicted Emoji: 😂


In [3]:
!pip install nbstripout
!nbstripout emoji_prediction.ipynb


Could not strip 'emoji_prediction.ipynb': file not found
